区块链网络是去中心化的，区块链不是基于一个中心节点产生的，而是由很多去中心化的节点一起参与维护的
我们要实现一个简单地去中心化网络，该模拟实验，多个节点可以运行在同一个计算机上，只是每个节点使用了不同的本地端口号，每个节点都是使用一个独立的线程运行，相当于一个独立的节点。

In [7]:
# 导入椭圆曲线算法
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError
import binascii
import base64
from hashlib import sha256


class Wallet:
    """
        钱包
    """
    def __init__(self):
        """
            钱包初始化时基于椭圆曲线生成一个唯一的秘钥对，代表区块链上一个唯一的账户
        """
        self._private_key = SigningKey.generate(curve=SECP256k1)
        self._public_key = self._private_key.get_verifying_key()

    @property
    def address(self):
        """
            这里通过公钥生成地址
        """
        h = sha256(self._public_key.to_pem())
        return base64.b64encode(h.digest())

    @property
    def pubkey(self):
        """
            返回公钥字符串
        """
        return self._public_key.to_pem()

    def sign(self, message):
        """
            生成数字签名
        """
        h = sha256(message.encode('utf8'))
        return binascii.hexlify(self._private_key.sign(h.digest()))

    
def verify_sign(pubkey, message, signature):
    """
        验证签名
    """
    verifier = VerifyingKey.from_pem(pubkey)
    h = sha256(message.encode('utf8'))
    return verifier.verify(binascii.unhexlify(signature), h.digest())

In [8]:
import json

class Transaction:
    """
    交易的结构
    """
    def __init__(self, sender, recipient, amount):
        """
            初始化交易，设置交易的发送方、接收方和交易数量
        """
        if isinstance(sender, bytes):
            sender = sender.decode('utf-8')
        self.sender = sender            # 发送方
        if isinstance(recipient, bytes):
            recipient = recipient.decode('utf-8')
        self.recipient = recipient      # 接收方
        self.amount = amount            # 交易数量
        
    def set_sign(self, signature, pubkey):
        """
            为了便于验证这个交易的可靠性，需要发送方输入他的公钥和签名
        """
        self.signature = signature      # 签名
        self.pubkey = pubkey            # 发送方公钥
        
    def __repr__(self):
        """
            交易大致可分为两种，一是挖矿所得，而是转账交易
            挖矿所得无发送方，以此进行区分显示不同内容
        """
        if self.sender:
            s = "从 %s 转至 %s %d个加密货币" % (self.sender, self.recipient, self.amount)
        else:
            s = "%s 挖矿获取%d个加密货币" % (self.recipient, self.amount)
        return s


class TransactionEncoder(json.JSONEncoder):
    """
    定义Json的编码类，用来序列化Transaction
    """
    def default(self, obj):
        if isinstance(obj, Transaction):
            return obj.__dict__
        else:
            return json.JSONEncoder.default(self, obj)

In [9]:
import hashlib
from datetime import datetime


class Block:
    """
        区块结构
            prev_hash:      父区块哈希值
            transactions:           交易对
            timestamp:      区块创建时间
            hash:           区块哈希值
            Nonce:        随机数
    """
    def __init__(self, transactions, prev_hash):
        # 将传入的父哈希值和数据保存到类变量中
        self.prev_hash = prev_hash    
        self.transactions = transactions
        # 获取当前时间
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 设置Nonce和哈希的初始值为None
        self.nonce = None
        self.hash = None
        
    def __repr__(self):
        return "区块内容：%s\n哈希值: %s" % (json.dumps(self.transactions), self.hash)

In [10]:
DIFFICULTY = 5

class ProofOfWork:
    """
        工作量证明
    """
    def __init__(self, block, miner, difficult=5):
        self.block = block
        
        # 定义工作量难度，默认为5，表示有效的哈希值以5个“0”开头
        self.difficulty = DIFFICULTY

        self.miner = miner
        # 添加挖矿奖励
        self.reward_amount = 1

    def mine(self):
        """
            挖矿函数
        """
        i = 0
        prefix = '0' * self.difficulty
        
        
        # 添加奖励
        t = Transaction(
                sender="",
                recipient=self.miner.address,
                amount=self.reward_amount,
            )
        sig = self.miner.sign(json.dumps(t, cls=TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)

        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            # 更新区块中的交易数据
            # message.update(str(self.block.data).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode("utf-8"))
            digest = message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce = i
                self.block.hash = digest
                return self.block
            i += 1

In [11]:
class BlockChain:
    """
        区块链结构体
            blocks:        包含的区块列表
    """
    def __init__(self):
        self.blocks = []

    def add_block(self, block):
        """
        添加区块
        """
        self.blocks.append(block)

# 新增内容：构建去中心化网络

In [12]:
import socket      #套接字，利用三元组【ip地址，协议，端口】可以进行网络间通信
import threading   #线程
import pickle

# 定义一个全局列表保存所有节点
NODE_LIST = []


class Node(threading.Thread):  #继承与线程
    def __init__(self, name, port, host="localhost"):
        threading.Thread.__init__(self, name=name)
        self.host = host           #  服务器地址，本地电脑都设为localhost
        self.port = port           # 每个节点对应一个唯一的端口号
        self.name = name           # 唯一的节点名称
        self.wallet = Wallet()
        self.blockchain = None    # 用来存储一个区块链副本  账本
        
    def run(self):
        """
            节点运行
        """
        self.init_blockchain()    # 初始化区块链
        
        # 在指定端口进行监听
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)      #获取TCP/ip  套接字sock
        sock.bind((self.host, self.port))                             #绑定主机，端口号到套接字sock
        NODE_LIST.append({
            "name": self.name,
            "host": self.host,
            "port": self.port
        })
        
        sock.listen(10)                                            #开始TCP监听
        print(self.name, "运行中...")
        
        while True:       # 不断处理其他节点发送的请求
            connection,address = sock.accept()                    #被动接受TCP客户的连接，(阻塞式)等待连接的到来
            try:
                print(self.name, "处理请求内容...")
                self.handle_request(connection)
            except socket.timeout:  
                print('超时!')
            except Exception as e:
                print(e, )
            connection.close()
            
            
            
            
    
    def handle_request(self, connection):
        data = []
        
        while True:    # 不断读取请求数据直至读取完成
            buf = connection.recv(1024)  
            if not buf: # 若读取不到新的数据则退出
                break
            data.append(buf)
            if len(buf) < 1024:   # 若读取到的数据长度小于规定长度，说明数据读取完成，退出
                break
        t = pickle.loads(b''.join(data)) #从pickle格式的文件中读取数据并转换为Python的类型。
        print("数据接受完成,判断数据的  类 ：交易，区块，初始化" )
        
        if isinstance(t, Transaction):  # 如果t是新交易类  消息
            print("处理交易请求...")
            if verify_sign(t.pubkey, str(t), t.signature):   #验证交易，公钥验证签名
                # 验证交易签名没问题，生成一个新的区块
                print(self.name, "验证交易成功")
            
                new_block = Block(transactions=[t], prev_hash="")
                print(self.name, "生成新的区块...")
                
                w = ProofOfWork(new_block, self.wallet)
                block = w.mine()                              #挖矿，挖到正确的区块哈希值，此处block就是新的区块，主要是找到了符合要求的nonce值
                print(self.name, "将新区块添加到区块链中")
                self.blockchain.add_block(block)
                
                
                print(self.name, "将新区块广播到网络中...")
                self.broadcast_new_block(block)
                
                
            else:
                print(self.name, "交易验证失败！")   #签名不对
                
        elif isinstance(t, Block):   #如果t是新区块类  消息
            print("处理新区块请求...")
            if self.verify_block(t):  
                print(self.name, "区块验证成功")  
                self.blockchain.add_block(t)
                print(self.name, "添加新区块成功")
            else:
                print(self.name, "区块验证失败!")
                
                
        else:  # 如果不是新区块消息，默认为初始化消息类型，返回本地区块链内容
            print("是我是我，我是初始化，我要返回我的区块链信息")
            connection.send(pickle.dumps(self.blockchain))
            
    def verify_block(self, block):
        """
            验证区块有效性   是否是符合难度的区块哈希值，找到了正确的nonce值
        """
        message = hashlib.sha256()
        message.update(str(block.prev_hash).encode('utf-8'))
        # 更新区块中的交易数据
        # message.update(str(self.block.data).encode('utf-8'))
        message.update(str(block.transactions).encode('utf-8'))
        message.update(str(block.timestamp).encode('utf-8'))
        message.update(str(block.nonce).encode('utf-8'))
        digest = message.hexdigest()

        prefix = '0' * DIFFICULTY
        return digest.startswith(prefix)
            
    def broadcast_new_block(self, block):
        """
            将新生成的区块广播到网络中其他节点
        """
        for node in NODE_LIST:     #遍历节点中的每一个节点，把新的区块广播给除了自己的所有节点
            host =node['host']
            port = node['port']
            
            if host == self.host and port == self.port:
                print(self.name, "忽略自身节点")
            else:
                print(self.name, "广播新区块至 %s" % (node['name']))
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.connect((host, port))    # 连接到网络中的节点
                sock.send(pickle.dumps(block))   # 发送新区块
                sock.close()          # 发送完成后关闭连接
                
    def init_blockchain(self):
        """
            初始化当前节点的区块链
        """
        #PER_BYTE = 1024
        if NODE_LIST:                # 若当前网络中已存在其他节点，则从第一个节点从获取区块链信息
            host = NODE_LIST[0]['host']
            port = NODE_LIST[0]['port']
            name = NODE_LIST[0]["name"]
            print(self.name, "发送初始化请求 %s" % (name))
            print("开始让节点1发送请求")
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)    #获取TCP/ip套接字
            sock.connect((host, port))    # 连接到网络中的第一个节点
            sock.send(pickle.dumps('INIT'))   # 发送初始化请求   pickle.dumps将Python数据转换为pickle格式的bytes字串
            print("请求成功")
            
            data = []
            print("开始接受节点1的connect返回的信息")
            while True:          # 读取区块链信息，直至完全获取后退出
                buf = sock.recv(1024)
                print("接收中")
                if not buf:
                    print("接收完毕，接空")
                    break
                
                data.append(buf)
                if len(buf) < 1024:
                    print("太短了，完毕")
                    break
            sock.close()   # 获取完成后关闭连接
            
            # 将获取的区块链信息赋值到当前节点
            self.blockchain = pickle.loads(b''.join(data))
            print(self.name, "初始化完成.")
            
        else:
            # 如果是网络中的第一个节点，初始化一个创世区块
            block = Block(transactions=[], prev_hash="")
            w = ProofOfWork(block, self.wallet)
            genesis_block = w.mine()
            self.blockchain = BlockChain()
            self.blockchain.add_block(genesis_block)
            print("生成创世区块")
    
    def submit_transaction(self, transaction):   #遍历节点中的每一个节点，把新的交易广播给除了自己的所有节点
         for node in NODE_LIST:
            host =node['host']
            port = node['port']
            
            if host == self.host and port == self.port:
                print(self.name, "忽略自身节点")
            else:
                print(self.name, "广播新区块至 %s:%s" % (self.host, self.port))
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
                sock.connect((node["host"], node["port"]))  
                sock.send(pickle.dumps(transaction)) 
                sock.close()

        
    def get_balance(self):
        balance = 0
        for block in self.blockchain.blocks:
            for t in block.transactions:
                if t.sender == self.wallet.address.decode():
                    balance -= t.amount
                elif t.recipient == self.wallet.address.decode():
                    balance += t.amount
        print("当前拥有%.1f个加密货币" % (balance))
    
    def print_blockchain(self):
        print("区块链包含区块个数: %d\n" % len(self.blockchain.blocks))
        for block in self.blockchain.blocks:
            print("上个区块哈希：%s" % block.prev_hash)
            print("区块内容：%s" % block.transactions)
            print("区块哈希：%s" % block.hash)
            print("\n") 

In [13]:
# 初始化节点1

node1 = Node("节点1", 8000)

In [14]:
node1.start()   #启动线程  调用 start() 方法是用来启动线程的，轮到该线程执行时，会自动调用 run()；直接调用 run() 方法

生成创世区块
节点1 运行中...


In [15]:
node1.print_blockchain()  #输出区块信息

区块链包含区块个数: 1

上个区块哈希：
区块内容：[ErjI8Y9VOsGqIiL/0VQry93kHrshx1vYifdUFIWNAUc= 挖矿获取1个加密货币]
区块哈希：000006874da9ea6f7d6d9c001785802266e68ba93cfaab4222951b80a6391f34




In [16]:
node2 = Node("节点2", 8001)

In [17]:
node2.start() 

节点2 发送初始化请求 节点1
开始让节点1发送请求
请求成功
开始接受节点1的connect返回的信息
节点1 处理请求内容...
数据接受完成,判断数据的  类 ：交易，区块，初始化
是我是我，我是初始化，我要返回我的区块链信息
接收中
太短了，完毕
节点2 初始化完成.
节点2 运行中...


In [18]:
node2.print_blockchain()

区块链包含区块个数: 1

上个区块哈希：
区块内容：[ErjI8Y9VOsGqIiL/0VQry93kHrshx1vYifdUFIWNAUc= 挖矿获取1个加密货币]
区块哈希：000006874da9ea6f7d6d9c001785802266e68ba93cfaab4222951b80a6391f34




In [19]:
node1.get_balance()

当前拥有1.0个加密货币


In [20]:
node2.get_balance()

当前拥有0.0个加密货币


In [21]:
#创建交易
new_transaction = Transaction(
    sender=node1.wallet.address,
    recipient=node2.wallet.address,
    amount=0.3
)
sig = node1.wallet.sign(str(new_transaction))  #私钥签名
new_transaction.set_sign(sig, node1.wallet.pubkey)#发送公钥，和签名，给验证者验证

In [22]:
node1.submit_transaction(new_transaction)    #广播交易

节点1 忽略自身节点
节点1 广播新区块至 localhost:8000
节点2 处理请求内容...
数据接受完成,判断数据的  类 ：交易，区块，初始化
处理交易请求...
节点2 验证交易成功
节点2 生成新的区块...
节点2 将新区块添加到区块链中
节点2 将新区块广播到网络中...
节点2 广播新区块至 节点1
节点1 处理请求内容...
节点2 忽略自身节点
数据接受完成,判断数据的  类 ：交易，区块，初始化
处理新区块请求...
节点1 区块验证成功
节点1 添加新区块成功


In [23]:
node1.print_blockchain()

区块链包含区块个数: 2

上个区块哈希：
区块内容：[ErjI8Y9VOsGqIiL/0VQry93kHrshx1vYifdUFIWNAUc= 挖矿获取1个加密货币]
区块哈希：000006874da9ea6f7d6d9c001785802266e68ba93cfaab4222951b80a6391f34


上个区块哈希：
区块内容：[从 ErjI8Y9VOsGqIiL/0VQry93kHrshx1vYifdUFIWNAUc= 转至 Zda6Vkd5lLMafNs+hBgm7oP9dtqRXLlHn/Q6os+OAPY= 0个加密货币, Zda6Vkd5lLMafNs+hBgm7oP9dtqRXLlHn/Q6os+OAPY= 挖矿获取1个加密货币]
区块哈希：000001f82209cae450a7745865dd0ed89d7792ecd0c438eaef9fa1100131a7db




In [24]:
node2.print_blockchain()

区块链包含区块个数: 2

上个区块哈希：
区块内容：[ErjI8Y9VOsGqIiL/0VQry93kHrshx1vYifdUFIWNAUc= 挖矿获取1个加密货币]
区块哈希：000006874da9ea6f7d6d9c001785802266e68ba93cfaab4222951b80a6391f34


上个区块哈希：
区块内容：[从 ErjI8Y9VOsGqIiL/0VQry93kHrshx1vYifdUFIWNAUc= 转至 Zda6Vkd5lLMafNs+hBgm7oP9dtqRXLlHn/Q6os+OAPY= 0个加密货币, Zda6Vkd5lLMafNs+hBgm7oP9dtqRXLlHn/Q6os+OAPY= 挖矿获取1个加密货币]
区块哈希：000001f82209cae450a7745865dd0ed89d7792ecd0c438eaef9fa1100131a7db




In [25]:
node1.get_balance()

当前拥有0.7个加密货币


In [26]:
node2.get_balance()

当前拥有1.3个加密货币
